## 1. Récupération de la DB et schéma de base de donnée 


Récupérer la base de données `blog.sqlite`

In [3]:
import sqlite3

In [4]:
conn = sqlite3.connect('./blog.sqlite')
cur = conn.cursor()

![Texte alternatif](bdd.png)

Réaliser un schéma de base de donnée (diagramme entité-relation (ERD)) à l'aide d'un outil, par exemple <a href='https://www.dbdesigner.net/' target="_blank">DB designer </a>

---
## 2. Retourner le top 3 des articles les plus liké.


In [3]:
query = """
        SELECT posts.title 
        FROM posts 
        JOIN likes ON posts.id = likes.post_id 
        GROUP BY posts.title 
        ORDER BY COUNT(likes.post_id) DESC
        LIMIT 3;
        """
print(cur.execute(query).fetchall())

[('Half imagine another.',), ('Side foot leader popular.',), ('Area paper whatever mean.',)]


---
## 3. Trouvez les trois utilisateurs qui ont le plus liké

In [4]:
query = """
        SELECT users.first_name || ' ' || users.last_name AS name 
        FROM users 
        JOIN likes ON users.id = likes.user_id 
        GROUP BY users.id 
        ORDER BY COUNT(likes.user_id) DESC 
        LIMIT 3;
        """
print(cur.execute(query).fetchall())

[('Michael Allen',), ('Donna Ramirez',), ('Hayley Williams',)]


---
## 4. Trouver l'auteur le plus apprécié

In [6]:
query = """
        SELECT users.first_name || ' ' || users.last_name AS name 
        FROM users 
        JOIN likes ON users.id = likes.user_id 
        GROUP BY likes.post_id
        ORDER BY COUNT(likes.user_id) DESC;
        """
print(cur.execute(query).fetchone())

('Bradley Kelly',)


---
## 5. Qui sont les auteurs des 3 articles les plus likés ?

In [7]:
query = """
        SELECT users.first_name || ' ' || users.last_name AS name 
        FROM users 
        JOIN likes ON users.id = likes.user_id 
        GROUP BY likes.post_id 
        ORDER BY COUNT(likes.user_id) DESC 
        LIMIT 3;
        """
print(cur.execute(query).fetchall())

[('Bradley Kelly',), ('Daniel Booth',), ('Cory Bush',)]


---
## 6. Combien de personnes ont aimé au moins une publication ?

In [9]:
query = """
        SELECT SUM(subquery.user_count) as users_count 
        FROM (
            SELECT COUNT(DISTINCT user_id) as user_count FROM likes
            ) 
        AS subquery;"""
print(cur.execute(query).fetchone())

(49,)


---
## 7. Calculez le nombre cumulé de likes par jour

In [17]:
query = """
        SELECT l1.created_at,
        (SELECT SUM(l2.id) FROM likes AS l2 WHERE l2.created_at <= l1.created_at AND l2.created_at = l1.created_at) AS cumul_likes
        FROM likes as l1
        GROUP BY l1.created_at
        ORDER BY l1.created_at ASC;
        """
print(cur.execute(query).fetchall())

[('2019-01-01', 124124), ('2019-01-02', 175187), ('2019-01-03', 221761), ('2019-01-04', 193885), ('2019-01-05', 136956), ('2019-01-06', 79699), ('2019-01-07', 135173), ('2019-01-08', 135540), ('2019-01-09', 195793), ('2019-01-10', 135089), ('2019-01-11', 128399), ('2019-01-12', 150020), ('2019-01-13', 184713), ('2019-01-14', 117433), ('2019-01-15', 138465), ('2019-01-16', 171619), ('2019-01-17', 184516), ('2019-01-18', 95175), ('2019-01-19', 99352), ('2019-01-20', 134700), ('2019-01-21', 127762), ('2019-01-22', 118977), ('2019-01-23', 168496), ('2019-01-24', 127696), ('2019-01-25', 147046), ('2019-01-26', 190169), ('2019-01-27', 170136), ('2019-01-28', 144503), ('2019-02-01', 104743), ('2019-02-02', 117425), ('2019-02-03', 174830), ('2019-02-04', 158164), ('2019-02-05', 149098), ('2019-02-06', 148829), ('2019-02-07', 119447), ('2019-02-08', 153564), ('2019-02-09', 115352), ('2019-02-10', 135232), ('2019-02-11', 171066), ('2019-02-12', 102727), ('2019-02-13', 143839), ('2019-02-14', 176

---
## 8. (Optional) Qui est le(s) plus grand(s) fan(s) de chaque auteur ?

<p>
Le ou les plus grands fans d'un auteur sont définis comme le ou les utilisateurs qui ont le plus aimé les publications de l'auteur. c'est-à-dire que s'il y a égalité entre les fans qui ont tous deux aimé un auteur 20 fois, les deux fans doivent être renvoyés avec leur nombre de likes et l'auteur en question.</p>

<p>Indice : Vous pouvez utiliser <code>WITH</code></p>

In [34]:
# TOP1 ou TOP2 users for n post from 1 user_id for each user from posts table
query = """
WITH Classement AS (
    SELECT
        posts.user_id as author_id,
        likes.user_id,
        posts.id as post_id,
        SUM(likes.id) AS nb_likes,
        RANK() OVER (PARTITION BY posts.id, posts.user_id ORDER BY SUM(likes.id) DESC) as classement
    FROM
        likes 
    JOIN
        posts ON posts.id = likes.post_id
    GROUP BY 
        posts.user_id, likes.user_id, posts.id
)
SELECT
    author_id,
    post_id,
    nb_likes,
    user_id
FROM
    Classement
WHERE
    classement = 1;
"""
print(cur.execute(query).fetchall())

[(50, 1, 21973, 16), (50, 2, 16627, 9), (50, 3, 22817, 4), (50, 4, 25532, 33), (50, 5, 25912, 39), (51, 6, 22533, 24), (51, 7, 23951, 12), (51, 8, 23972, 46), (51, 9, 39491, 11), (51, 10, 22436, 11), (51, 11, 20358, 6), (51, 12, 25413, 21), (51, 13, 27676, 8), (51, 14, 27139, 34), (51, 15, 31206, 46), (52, 16, 28194, 3), (52, 17, 27653, 2), (52, 18, 24521, 44), (52, 19, 23813, 32), (52, 20, 24720, 31), (52, 21, 24080, 24), (52, 22, 19761, 21), (52, 23, 28453, 48), (53, 24, 24025, 34), (54, 25, 26332, 36), (54, 26, 28463, 44), (54, 27, 22450, 18), (55, 28, 23033, 27), (55, 29, 15618, 31), (55, 30, 29086, 29), (55, 31, 17331, 9), (55, 32, 20354, 33), (55, 33, 18833, 14), (55, 34, 29193, 18), (55, 35, 25989, 25), (56, 36, 24836, 38), (56, 37, 34182, 31), (56, 38, 26065, 29), (56, 39, 30711, 12), (57, 40, 23005, 42), (57, 41, 21552, 2), (57, 42, 32661, 2), (57, 43, 25914, 45), (57, 44, 45348, 27), (57, 45, 25823, 1), (57, 46, 26817, 39), (57, 47, 30484, 9), (57, 48, 25786, 49), (57, 49, 25